In [3]:
import numpy as np  
import pandas as pd 
import re   
import seaborn as sns 
import matplotlib.pyplot as plt     
from IPython.core.display import display, HTML  
import string
import tensorflow as tf
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences   
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Convolution1D, Flatten, LeakyReLU
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, GlobalAveragePooling1D, MaxPooling1D, GlobalMaxPooling1D,Embedding,SimpleRNN
from tensorflow.keras.layers import Concatenate
from tqdm import tqdm  
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras import callbacks
from tensorflow.keras.models import Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras.models import load_model
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import model_to_dot

In [4]:
#configure GPU to set memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#read in training data
train = pd.read_csv("/content/drive/MyDrive/MSCA Machine Learning/Data/cleaned_train.csv")
train_y = train.target

In [6]:
#tokenize training data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train.text)   
word_index = tokenizer.word_index    
num_words = len(tokenizer.word_index)+1
print('Number of unique words:',len(word_index))

Number of unique words: 17440


In [7]:
#Prepare training data for input to glove embedding
#setting standardized max length of 50
training_sequences = tokenizer.texts_to_sequences(train.text)  

training_padded = pad_sequences(training_sequences,                                  
                                   maxlen=50,                                      
                                   padding='pre',                           
                                   truncating='pre')  

X_train, X_test, Y_train, Y_test = train_test_split(training_padded, train_y, test_size=.25,random_state=0)

In [ ]:
#creating embedding dictionary
embedding_dict={}
with open('glove.6B.300d.txt','r',encoding='utf-8') as f:
    for line in f:
        values=line.split()
        word=values[0]
        vectors=np.asarray(values[1:],'float32')
        embedding_dict[word]=vectors
f.close()

embedding_dim=300
embedding_matrix = np.zeros((num_words, embedding_dim))
#Creating GloVe embeddings matrix
for word, i in tqdm(word_index.items()):
    if i < num_words:
        embedding_vector = embedding_dict.get(word)  
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

embedding_matrix.shape

100%|██████████| 17440/17440 [00:00<00:00, 256082.42it/s]


(17441, 300)

In [9]:
#saving embedding matrix
import pickle
with open('/content/drive/MyDrive/MSCA Machine Learning/Working Notebooks/glov_emb.pkl','rb')as f:
    embedding_matrix=pickle.load(f)

In [11]:
from tensorflow import keras
from kerastuner.tuners import RandomSearch
from kerastuner import RandomSearch, Hyperband
# Model Hyperparameters
embedding_dim = 300

sequence_length = 50

NUM_CLASSES = 1

In [ ]:
#initialize simple RNN model with hyperparameter options
def build_model(hp):
    strategy = tf.distribute.MirroredStrategy()
    with strategy.scope():
        model = Sequential()
        model.add(Embedding(input_dim=num_words,
                    output_dim=300,
                    embeddings_initializer=initializers.Constant(embedding_matrix), 
                    input_length=50,trainable=False))
        model.add(
            Dropout(
                rate=hp.Float(
                    "dropout_1", min_value=0.0, max_value=0.4, default=0.2, step=0.05,
                )
            )
        )
        model.add(            
            SimpleRNN(
                units = 64,
                activation = hp.Choice(
                    "dense_activation",
                    values=["relu", "tanh", "sigmoid"],
                    default="relu",
                )
            ))
        model.add(
            Dense(
                units=1, 
                activation="sigmoid"))

        model.compile(            
            optimizer=keras.optimizers.Adam(
                hp.Float(
                    "learning_rate",
                    min_value=1e-4,
                    max_value=1e-2,
                    sampling="LOG",
                    default=1e-3,
                )
            ),
            loss="binary_crossentropy",
            metrics=["accuracy"])
    return model

In [ ]:
#define random search to search to optimize hyperparameter in simple RNN
R_tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='ML-Project')

INFO:tensorflow:Reloading Oracle from existing project project/ML-Project/oracle.json
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [ ]:
#define hyperband to search to optimize hyperparameter in simple RNN
H_tuner = Hyperband(
    build_model,
    max_epochs=10,
    objective='val_accuracy',
    executions_per_trial=3,
    directory='project_hyperband',
    project_name='ML-Project')

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [ ]:
#use random search to search to optimize hyperparameter in simple RNN
R_tuner.search(X_train, Y_train,
             epochs=5,
             validation_data=(X_test, Y_test))
random_summary=R_tuner.results_summary()
random_summary

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Epoch 1/5
179/179 [==============================] - ETA: 0s - loss: 0.6773 - accuracy: 0.62 - ETA: 2s - loss: 0.6820 - accuracy: 0.60 - ETA: 2s - loss: 0.6701 - accuracy: 0.61 - ETA: 2s - loss: 0.6732 - accuracy: 0.60 - ETA: 2s - loss: 0.6821 - accuracy: 0.58 - ETA: 2s - loss: 0.6797 - accuracy: 0.58 - ETA: 2s - loss: 0.6736 - accuracy: 0.59 - ETA: 2s - loss: 0.6760 - accuracy: 0.58 - ETA: 2s - loss: 0.6734 - accuracy: 0.59 - ETA: 2s - loss: 0.6706 - accuracy: 0.59 - ETA: 2s - loss: 0.6698 - accuracy: 0.59 - ETA: 1s - loss: 0.6723 - accuracy: 0.59 - ETA: 1s - loss: 0.6706 - accuracy: 0.59 - ETA: 1s - loss: 0.6712 - accuracy: 0.59 - ETA: 1s - loss: 0.6685 - accuracy: 0.60 - ETA: 1s - loss: 0.6678 - accuracy: 0.60 - ETA: 1s - loss: 0.6689 - accuracy: 0.59 - ETA: 1s - loss: 0.6685 - accuracy: 0.60 - ETA: 1s - loss: 0.6696 - accuracy: 0.59 - ETA: 1s - loss: 0.6704 - accuracy: 0.59 - ETA: 

179/179 [==============================] - ETA: 0s - loss: 0.5871 - accuracy: 0.62 - ETA: 2s - loss: 0.5789 - accuracy: 0.69 - ETA: 2s - loss: 0.5993 - accuracy: 0.67 - ETA: 2s - loss: 0.6111 - accuracy: 0.65 - ETA: 2s - loss: 0.6230 - accuracy: 0.64 - ETA: 2s - loss: 0.6160 - accuracy: 0.65 - ETA: 2s - loss: 0.6141 - accuracy: 0.66 - ETA: 2s - loss: 0.6195 - accuracy: 0.66 - ETA: 2s - loss: 0.6152 - accuracy: 0.66 - ETA: 2s - loss: 0.6172 - accuracy: 0.66 - ETA: 2s - loss: 0.6153 - accuracy: 0.66 - ETA: 2s - loss: 0.6151 - accuracy: 0.66 - ETA: 2s - loss: 0.6146 - accuracy: 0.66 - ETA: 2s - loss: 0.6144 - accuracy: 0.66 - ETA: 2s - loss: 0.6101 - accuracy: 0.67 - ETA: 1s - loss: 0.6103 - accuracy: 0.67 - ETA: 1s - loss: 0.6082 - accuracy: 0.67 - ETA: 1s - loss: 0.6106 - accuracy: 0.67 - ETA: 1s - loss: 0.6097 - accuracy: 0.67 - ETA: 1s - loss: 0.6119 - accuracy: 0.67 - ETA: 1s - loss: 0.6101 - accuracy: 0.67 - ETA: 1s - loss: 0.6087 - accuracy: 0.67 - ETA: 1s - loss: 0.6081 - accuracy

179/179 [==============================] - ETA: 0s - loss: 0.6612 - accuracy: 0.59 - ETA: 1s - loss: 0.6420 - accuracy: 0.60 - ETA: 1s - loss: 0.6364 - accuracy: 0.62 - ETA: 1s - loss: 0.6483 - accuracy: 0.62 - ETA: 1s - loss: 0.6436 - accuracy: 0.63 - ETA: 1s - loss: 0.6466 - accuracy: 0.62 - ETA: 1s - loss: 0.6455 - accuracy: 0.61 - ETA: 1s - loss: 0.6435 - accuracy: 0.62 - ETA: 1s - loss: 0.6423 - accuracy: 0.61 - ETA: 1s - loss: 0.6350 - accuracy: 0.63 - ETA: 1s - loss: 0.6355 - accuracy: 0.62 - ETA: 1s - loss: 0.6344 - accuracy: 0.63 - ETA: 1s - loss: 0.6347 - accuracy: 0.63 - ETA: 1s - loss: 0.6354 - accuracy: 0.63 - ETA: 1s - loss: 0.6331 - accuracy: 0.63 - ETA: 1s - loss: 0.6339 - accuracy: 0.63 - ETA: 1s - loss: 0.6336 - accuracy: 0.63 - ETA: 1s - loss: 0.6323 - accuracy: 0.63 - ETA: 1s - loss: 0.6350 - accuracy: 0.63 - ETA: 1s - loss: 0.6351 - accuracy: 0.63 - ETA: 0s - loss: 0.6349 - accuracy: 0.63 - ETA: 0s - loss: 0.6349 - accuracy: 0.63 - ETA: 0s - loss: 0.6347 - accuracy

179/179 [==============================] - ETA: 0s - loss: 0.6601 - accuracy: 0.59 - ETA: 1s - loss: 0.6639 - accuracy: 0.59 - ETA: 1s - loss: 0.6631 - accuracy: 0.59 - ETA: 1s - loss: 0.6565 - accuracy: 0.60 - ETA: 1s - loss: 0.6526 - accuracy: 0.61 - ETA: 1s - loss: 0.6544 - accuracy: 0.61 - ETA: 1s - loss: 0.6544 - accuracy: 0.61 - ETA: 1s - loss: 0.6542 - accuracy: 0.61 - ETA: 1s - loss: 0.6557 - accuracy: 0.61 - ETA: 1s - loss: 0.6551 - accuracy: 0.61 - ETA: 1s - loss: 0.6544 - accuracy: 0.61 - ETA: 1s - loss: 0.6522 - accuracy: 0.62 - ETA: 1s - loss: 0.6543 - accuracy: 0.61 - ETA: 1s - loss: 0.6537 - accuracy: 0.61 - ETA: 1s - loss: 0.6518 - accuracy: 0.62 - ETA: 1s - loss: 0.6522 - accuracy: 0.62 - ETA: 1s - loss: 0.6532 - accuracy: 0.61 - ETA: 1s - loss: 0.6535 - accuracy: 0.61 - ETA: 1s - loss: 0.6520 - accuracy: 0.61 - ETA: 1s - loss: 0.6513 - accuracy: 0.62 - ETA: 0s - loss: 0.6512 - accuracy: 0.62 - ETA: 0s - loss: 0.6508 - accuracy: 0.61 - ETA: 0s - loss: 0.6509 - accuracy

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Epoch 1/5
179/179 [==============================] - ETA: 0s - loss: 0.7691 - accuracy: 0.53 - ETA: 1s - loss: 0.7462 - accuracy: 0.52 - ETA: 1s - loss: 0.7071 - accuracy: 0.57 - ETA: 1s - loss: 0.6985 - accuracy: 0.56 - ETA: 1s - loss: 0.6888 - accuracy: 0.56 - ETA: 1s - loss: 0.6785 - accuracy: 0.57 - ETA: 1s - loss: 0.6769 - accuracy: 0.57 - ETA: 1s - loss: 0.6772 - accuracy: 0.57 - ETA: 1s - loss: 0.6733 - accuracy: 0.57 - ETA: 1s - loss: 0.6764 - accuracy: 0.57 - ETA: 1s - loss: 0.6729 - accuracy: 0.58 - ETA: 1s - loss: 0.6710 - accuracy: 0.58 - ETA: 1s - loss: 0.6735 - accuracy: 0.58 - ETA: 1s - loss: 0.6717 - accuracy: 0.58 - ETA: 1s - loss: 0.6699 - accuracy: 0.58 - ETA: 1s - loss: 0.6661 - accuracy: 0.58 - ETA: 1s - loss: 0.6640 - accuracy: 0.59 - ETA: 1s - loss: 0.6614 - accuracy: 0.59 - ETA: 1s - loss: 0.6551 - accuracy: 0.60 - ETA: 1s - loss: 0.6525 - accuracy: 0.60 - ETA: 

...

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Epoch 1/5
179/179 [==============================] - ETA: 0s - loss: 0.5089 - accuracy: 0.81 - ETA: 1s - loss: 0.7607 - accuracy: 0.61 - ETA: 1s - loss: 0.7169 - accuracy: 0.61 - ETA: 1s - loss: 0.7090 - accuracy: 0.58 - ETA: 1s - loss: 0.7076 - accuracy: 0.56 - ETA: 1s - loss: 0.7018 - accuracy: 0.55 - ETA: 1s - loss: 0.6980 - accuracy: 0.56 - ETA: 1s - loss: 0.6895 - accuracy: 0.57 - ETA: 1s - loss: 0.6880 - accuracy: 0.57 - ETA: 1s - loss: 0.6871 - accuracy: 0.57 - ETA: 1s - loss: 0.6819 - accuracy: 0.58 - ETA: 1s - loss: 0.6799 - accuracy: 0.58 - ETA: 1s - loss: 0.6736 - accuracy: 0.59 - ETA: 1s - loss: 0.6717 - accuracy: 0.58 - ETA: 1s - loss: 0.6674 - accuracy: 0.59 - ETA: 1s - loss: 0.6657 - accuracy: 0.59 - ETA: 1s - loss: 0.6663 - accuracy: 0.59 - ETA: 1s - loss: 0.6640 - accuracy: 0.60 - ETA: 1s - loss: 0.6619 - accuracy: 0.60 - ETA: 1s - loss: 0.6630 - accuracy: 0.59 - ETA: 

179/179 [==============================] - ETA: 0s - loss: 0.2303 - accuracy: 0.93 - ETA: 1s - loss: 0.4056 - accuracy: 0.84 - ETA: 1s - loss: 0.4237 - accuracy: 0.83 - ETA: 1s - loss: 0.4446 - accuracy: 0.81 - ETA: 1s - loss: 0.4482 - accuracy: 0.81 - ETA: 1s - loss: 0.4409 - accuracy: 0.81 - ETA: 1s - loss: 0.4469 - accuracy: 0.81 - ETA: 1s - loss: 0.4402 - accuracy: 0.81 - ETA: 1s - loss: 0.4333 - accuracy: 0.82 - ETA: 1s - loss: 0.4360 - accuracy: 0.81 - ETA: 1s - loss: 0.4356 - accuracy: 0.81 - ETA: 1s - loss: 0.4389 - accuracy: 0.81 - ETA: 1s - loss: 0.4383 - accuracy: 0.81 - ETA: 1s - loss: 0.4430 - accuracy: 0.81 - ETA: 1s - loss: 0.4503 - accuracy: 0.80 - ETA: 1s - loss: 0.4447 - accuracy: 0.81 - ETA: 1s - loss: 0.4482 - accuracy: 0.81 - ETA: 1s - loss: 0.4489 - accuracy: 0.81 - ETA: 1s - loss: 0.4478 - accuracy: 0.81 - ETA: 1s - loss: 0.4492 - accuracy: 0.80 - ETA: 0s - loss: 0.4523 - accuracy: 0.80 - ETA: 0s - loss: 0.4496 - accuracy: 0.80 - ETA: 0s - loss: 0.4483 - accuracy

179/179 [==============================] - ETA: 0s - loss: 0.5570 - accuracy: 0.75 - ETA: 1s - loss: 0.4557 - accuracy: 0.78 - ETA: 1s - loss: 0.4423 - accuracy: 0.81 - ETA: 1s - loss: 0.4485 - accuracy: 0.81 - ETA: 1s - loss: 0.4520 - accuracy: 0.80 - ETA: 1s - loss: 0.4569 - accuracy: 0.80 - ETA: 1s - loss: 0.4496 - accuracy: 0.80 - ETA: 1s - loss: 0.4433 - accuracy: 0.81 - ETA: 1s - loss: 0.4415 - accuracy: 0.81 - ETA: 1s - loss: 0.4358 - accuracy: 0.81 - ETA: 1s - loss: 0.4375 - accuracy: 0.81 - ETA: 1s - loss: 0.4311 - accuracy: 0.82 - ETA: 1s - loss: 0.4300 - accuracy: 0.82 - ETA: 1s - loss: 0.4330 - accuracy: 0.81 - ETA: 1s - loss: 0.4373 - accuracy: 0.81 - ETA: 1s - loss: 0.4384 - accuracy: 0.81 - ETA: 1s - loss: 0.4371 - accuracy: 0.81 - ETA: 1s - loss: 0.4399 - accuracy: 0.81 - ETA: 1s - loss: 0.4387 - accuracy: 0.81 - ETA: 1s - loss: 0.4361 - accuracy: 0.81 - ETA: 0s - loss: 0.4358 - accuracy: 0.81 - ETA: 0s - loss: 0.4378 - accuracy: 0.81 - ETA: 0s - loss: 0.4406 - accuracy

179/179 [==============================] - ETA: 0s - loss: 0.5505 - accuracy: 0.71 - ETA: 1s - loss: 0.4759 - accuracy: 0.77 - ETA: 1s - loss: 0.4784 - accuracy: 0.77 - ETA: 1s - loss: 0.4630 - accuracy: 0.78 - ETA: 1s - loss: 0.4720 - accuracy: 0.77 - ETA: 1s - loss: 0.4660 - accuracy: 0.78 - ETA: 1s - loss: 0.4603 - accuracy: 0.78 - ETA: 1s - loss: 0.4566 - accuracy: 0.79 - ETA: 1s - loss: 0.4589 - accuracy: 0.79 - ETA: 1s - loss: 0.4717 - accuracy: 0.78 - ETA: 1s - loss: 0.4747 - accuracy: 0.78 - ETA: 1s - loss: 0.4738 - accuracy: 0.78 - ETA: 1s - loss: 0.4742 - accuracy: 0.78 - ETA: 1s - loss: 0.4714 - accuracy: 0.78 - ETA: 1s - loss: 0.4709 - accuracy: 0.78 - ETA: 1s - loss: 0.4691 - accuracy: 0.78 - ETA: 1s - loss: 0.4672 - accuracy: 0.78 - ETA: 1s - loss: 0.4693 - accuracy: 0.78 - ETA: 1s - loss: 0.4732 - accuracy: 0.78 - ETA: 1s - loss: 0.4752 - accuracy: 0.78 - ETA: 0s - loss: 0.4762 - accuracy: 0.78 - ETA: 0s - loss: 0.4781 - accuracy: 0.78 - ETA: 0s - loss: 0.4746 - accuracy

INFO:tensorflow:Oracle triggered exit


In [ ]:
#use hyperband to search to optimize hyperparameter in simple RNN
H_tuner.search(X_train, Y_train,
             epochs=5,
             validation_data=(X_test, Y_test))
Hyperband_summary=H_tuner.results_summary()
Hyperband_summary

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Epoch 1/2
179/179 [==============================] - ETA: 0s - loss: 0.7173 - accuracy: 0.43 - ETA: 1s - loss: 0.6813 - accuracy: 0.57 - ETA: 1s - loss: 0.6718 - accuracy: 0.61 - ETA: 1s - loss: 0.6742 - accuracy: 0.63 - ETA: 1s - loss: 0.6808 - accuracy: 0.63 - ETA: 1s - loss: 0.6722 - accuracy: 0.64 - ETA: 1s - loss: 0.6579 - accuracy: 0.65 - ETA: 1s - loss: 0.6503 - accuracy: 0.66 - ETA: 1s - loss: 0.6362 - accuracy: 0.67 - ETA: 1s - loss: 0.6398 - accuracy: 0.67 - ETA: 1s - loss: 0.6314 - accuracy: 0.68 - ETA: 1s - loss: 0.6143 - accuracy: 0.69 - ETA: 1s - loss: 0.6114 - accuracy: 0.70 - ETA: 1s - loss: 0.6045 - accuracy: 0.70 - ETA: 1s - loss: 0.5951 - accuracy: 0.71 - ETA: 1s - loss: 0.5978 - accuracy: 0.71 - ETA: 1s - loss: 0.5978 - accuracy: 0.71 - ETA: 1s - loss: 0.5975 - accuracy: 0.71 - ETA: 1s - loss: 0.5953 - accuracy: 0.71 - ETA: 1s - loss: 0.5897 - accuracy: 0.71 - ETA: 

179/179 [==============================] - ETA: 0s - loss: 0.9151 - accuracy: 0.46 - ETA: 1s - loss: 0.7732 - accuracy: 0.54 - ETA: 1s - loss: 0.6918 - accuracy: 0.61 - ETA: 1s - loss: 0.6820 - accuracy: 0.62 - ETA: 1s - loss: 0.6573 - accuracy: 0.64 - ETA: 1s - loss: 0.6499 - accuracy: 0.65 - ETA: 1s - loss: 0.6464 - accuracy: 0.66 - ETA: 1s - loss: 0.6313 - accuracy: 0.67 - ETA: 1s - loss: 0.6349 - accuracy: 0.67 - ETA: 1s - loss: 0.6337 - accuracy: 0.67 - ETA: 1s - loss: 0.6235 - accuracy: 0.68 - ETA: 1s - loss: 0.6125 - accuracy: 0.69 - ETA: 1s - loss: 0.6120 - accuracy: 0.69 - ETA: 1s - loss: 0.6047 - accuracy: 0.70 - ETA: 1s - loss: 0.6026 - accuracy: 0.70 - ETA: 1s - loss: 0.5997 - accuracy: 0.70 - ETA: 1s - loss: 0.5946 - accuracy: 0.71 - ETA: 1s - loss: 0.5877 - accuracy: 0.71 - ETA: 1s - loss: 0.5876 - accuracy: 0.71 - ETA: 1s - loss: 0.5843 - accuracy: 0.71 - ETA: 0s - loss: 0.5834 - accuracy: 0.72 - ETA: 0s - loss: 0.5829 - accuracy: 0.72 - ETA: 0s - loss: 0.5862 - accuracy

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Epoch 1/2
179/179 [==============================] - ETA: 0s - loss: 0.7114 - accuracy: 0.59 - ETA: 1s - loss: 0.7090 - accuracy: 0.51 - ETA: 1s - loss: 0.6930 - accuracy: 0.56 - ETA: 1s - loss: 0.6959 - accuracy: 0.56 - ETA: 1s - loss: 0.6890 - accuracy: 0.57 - ETA: 1s - loss: 0.6900 - accuracy: 0.58 - ETA: 1s - loss: 0.6881 - accuracy: 0.58 - ETA: 1s - loss: 0.6861 - accuracy: 0.59 - ETA: 1s - loss: 0.6803 - accuracy: 0.59 - ETA: 1s - loss: 0.6779 - accuracy: 0.60 - ETA: 1s - loss: 0.6743 - accuracy: 0.60 - ETA: 1s - loss: 0.6724 - accuracy: 0.60 - ETA: 1s - loss: 0.6691 - accuracy: 0.61 - ETA: 1s - loss: 0.6640 - accuracy: 0.61 - ETA: 1s - loss: 0.6599 - accuracy: 0.62 - ETA: 1s - loss: 0.6541 - accuracy: 0.62 - ETA: 1s - loss: 0.6514 - accuracy: 0.63 - ETA: 1s - loss: 0.6466 - accuracy: 0.63 - ETA: 1s - loss: 0.6438 - accuracy: 0.63 - ETA: 1s - loss: 0.6391 - accuracy: 0.64 - ETA: 

179/179 [==============================] - ETA: 0s - loss: 0.7883 - accuracy: 0.46 - ETA: 1s - loss: 0.7442 - accuracy: 0.51 - ETA: 1s - loss: 0.7429 - accuracy: 0.52 - ETA: 1s - loss: 0.7633 - accuracy: 0.51 - ETA: 1s - loss: 0.7584 - accuracy: 0.51 - ETA: 1s - loss: 0.7450 - accuracy: 0.53 - ETA: 1s - loss: 0.7354 - accuracy: 0.55 - ETA: 1s - loss: 0.7284 - accuracy: 0.55 - ETA: 1s - loss: 0.7209 - accuracy: 0.55 - ETA: 1s - loss: 0.7140 - accuracy: 0.56 - ETA: 1s - loss: 0.7102 - accuracy: 0.56 - ETA: 1s - loss: 0.6998 - accuracy: 0.57 - ETA: 1s - loss: 0.6911 - accuracy: 0.58 - ETA: 1s - loss: 0.6846 - accuracy: 0.59 - ETA: 1s - loss: 0.6784 - accuracy: 0.59 - ETA: 1s - loss: 0.6720 - accuracy: 0.60 - ETA: 1s - loss: 0.6657 - accuracy: 0.60 - ETA: 1s - loss: 0.6589 - accuracy: 0.61 - ETA: 1s - loss: 0.6528 - accuracy: 0.62 - ETA: 1s - loss: 0.6417 - accuracy: 0.63 - ETA: 0s - loss: 0.6375 - accuracy: 0.63 - ETA: 0s - loss: 0.6360 - accuracy: 0.64 - ETA: 0s - loss: 0.6329 - accuracy

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Epoch 1/2
179/179 [==============================] - ETA: 0s - loss: 0.7258 - accuracy: 0.56 - ETA: 1s - loss: 0.7324 - accuracy: 0.55 - ETA: 1s - loss: 0.7158 - accuracy: 0.56 - ETA: 1s - loss: 0.7270 - accuracy: 0.54 - ETA: 1s - loss: 0.7256 - accuracy: 0.54 - ETA: 1s - loss: 0.7224 - accuracy: 0.55 - ETA: 1s - loss: 0.7269 - accuracy: 0.54 - ETA: 1s - loss: 0.7201 - accuracy: 0.55 - ETA: 1s - loss: 0.7265 - accuracy: 0.54 - ETA: 1s - loss: 0.7180 - accuracy: 0.55 - ETA: 1s - loss: 0.7125 - accuracy: 0.56 - ETA: 1s - loss: 0.7087 - accuracy: 0.56 - ETA: 1s - loss: 0.7060 - accuracy: 0.57 - ETA: 1s - loss: 0.6967 - accuracy: 0.58 - ETA: 1s - loss: 0.6893 - accuracy: 0.58 - ETA: 1s - loss: 0.6811 - accuracy: 0.59 - ETA: 1s - loss: 0.6734 - accuracy: 0.60 - ETA: 1s - loss: 0.6660 - accuracy: 0.61 - ETA: 1s - loss: 0.6607 - accuracy: 0.61 - ETA: 1s - loss: 0.6559 - accuracy: 0.62 - ETA: 

...

179/179 [==============================] - ETA: 0s - loss: 0.4386 - accuracy: 0.78 - ETA: 1s - loss: 0.4617 - accuracy: 0.78 - ETA: 1s - loss: 0.4612 - accuracy: 0.80 - ETA: 1s - loss: 0.4606 - accuracy: 0.80 - ETA: 1s - loss: 0.4575 - accuracy: 0.80 - ETA: 1s - loss: 0.4587 - accuracy: 0.80 - ETA: 1s - loss: 0.4586 - accuracy: 0.80 - ETA: 1s - loss: 0.4579 - accuracy: 0.80 - ETA: 1s - loss: 0.4633 - accuracy: 0.79 - ETA: 1s - loss: 0.4665 - accuracy: 0.79 - ETA: 1s - loss: 0.4674 - accuracy: 0.79 - ETA: 1s - loss: 0.4721 - accuracy: 0.79 - ETA: 1s - loss: 0.4676 - accuracy: 0.79 - ETA: 1s - loss: 0.4627 - accuracy: 0.79 - ETA: 1s - loss: 0.4598 - accuracy: 0.80 - ETA: 1s - loss: 0.4631 - accuracy: 0.79 - ETA: 1s - loss: 0.4630 - accuracy: 0.79 - ETA: 1s - loss: 0.4644 - accuracy: 0.79 - ETA: 1s - loss: 0.4616 - accuracy: 0.79 - ETA: 1s - loss: 0.4597 - accuracy: 0.79 - ETA: 1s - loss: 0.4586 - accuracy: 0.79 - ETA: 0s - loss: 0.4556 - accuracy: 0.80 - ETA: 0s - loss: 0.4535 - accuracy

179/179 [==============================] - ETA: 0s - loss: 0.8620 - accuracy: 0.40 - ETA: 1s - loss: 0.7531 - accuracy: 0.44 - ETA: 1s - loss: 0.7078 - accuracy: 0.53 - ETA: 1s - loss: 0.7190 - accuracy: 0.52 - ETA: 1s - loss: 0.7121 - accuracy: 0.52 - ETA: 1s - loss: 0.7049 - accuracy: 0.52 - ETA: 1s - loss: 0.6980 - accuracy: 0.53 - ETA: 1s - loss: 0.6950 - accuracy: 0.53 - ETA: 1s - loss: 0.6876 - accuracy: 0.55 - ETA: 1s - loss: 0.6816 - accuracy: 0.56 - ETA: 1s - loss: 0.6774 - accuracy: 0.57 - ETA: 1s - loss: 0.6772 - accuracy: 0.57 - ETA: 1s - loss: 0.6758 - accuracy: 0.57 - ETA: 1s - loss: 0.6744 - accuracy: 0.58 - ETA: 1s - loss: 0.6736 - accuracy: 0.58 - ETA: 1s - loss: 0.6714 - accuracy: 0.58 - ETA: 1s - loss: 0.6711 - accuracy: 0.58 - ETA: 1s - loss: 0.6709 - accuracy: 0.58 - ETA: 1s - loss: 0.6699 - accuracy: 0.58 - ETA: 1s - loss: 0.6706 - accuracy: 0.58 - ETA: 0s - loss: 0.6685 - accuracy: 0.59 - ETA: 0s - loss: 0.6676 - accuracy: 0.59 - ETA: 0s - loss: 0.6672 - accuracy

179/179 [==============================] - ETA: 0s - loss: 0.4417 - accuracy: 0.87 - ETA: 1s - loss: 0.4850 - accuracy: 0.78 - ETA: 2s - loss: 0.4938 - accuracy: 0.78 - ETA: 2s - loss: 0.4632 - accuracy: 0.80 - ETA: 2s - loss: 0.4660 - accuracy: 0.80 - ETA: 2s - loss: 0.4608 - accuracy: 0.80 - ETA: 1s - loss: 0.4583 - accuracy: 0.80 - ETA: 1s - loss: 0.4554 - accuracy: 0.80 - ETA: 1s - loss: 0.4583 - accuracy: 0.80 - ETA: 1s - loss: 0.4582 - accuracy: 0.79 - ETA: 1s - loss: 0.4618 - accuracy: 0.79 - ETA: 1s - loss: 0.4628 - accuracy: 0.79 - ETA: 1s - loss: 0.4682 - accuracy: 0.78 - ETA: 1s - loss: 0.4719 - accuracy: 0.78 - ETA: 1s - loss: 0.4726 - accuracy: 0.78 - ETA: 1s - loss: 0.4733 - accuracy: 0.78 - ETA: 1s - loss: 0.4716 - accuracy: 0.78 - ETA: 1s - loss: 0.4687 - accuracy: 0.78 - ETA: 1s - loss: 0.4679 - accuracy: 0.78 - ETA: 1s - loss: 0.4678 - accuracy: 0.78 - ETA: 1s - loss: 0.4643 - accuracy: 0.78 - ETA: 1s - loss: 0.4663 - accuracy: 0.78 - ETA: 1s - loss: 0.4641 - accuracy

179/179 [==============================] - ETA: 0s - loss: 0.4388 - accuracy: 0.78 - ETA: 2s - loss: 0.5428 - accuracy: 0.75 - ETA: 2s - loss: 0.4763 - accuracy: 0.78 - ETA: 2s - loss: 0.4672 - accuracy: 0.78 - ETA: 2s - loss: 0.4404 - accuracy: 0.80 - ETA: 2s - loss: 0.4367 - accuracy: 0.81 - ETA: 2s - loss: 0.4428 - accuracy: 0.80 - ETA: 2s - loss: 0.4382 - accuracy: 0.81 - ETA: 2s - loss: 0.4319 - accuracy: 0.81 - ETA: 2s - loss: 0.4445 - accuracy: 0.80 - ETA: 2s - loss: 0.4415 - accuracy: 0.80 - ETA: 2s - loss: 0.4451 - accuracy: 0.80 - ETA: 1s - loss: 0.4478 - accuracy: 0.80 - ETA: 1s - loss: 0.4468 - accuracy: 0.80 - ETA: 1s - loss: 0.4459 - accuracy: 0.80 - ETA: 1s - loss: 0.4431 - accuracy: 0.80 - ETA: 1s - loss: 0.4422 - accuracy: 0.80 - ETA: 1s - loss: 0.4428 - accuracy: 0.80 - ETA: 1s - loss: 0.4448 - accuracy: 0.80 - ETA: 1s - loss: 0.4434 - accuracy: 0.80 - ETA: 1s - loss: 0.4431 - accuracy: 0.80 - ETA: 1s - loss: 0.4445 - accuracy: 0.80 - ETA: 1s - loss: 0.4449 - accuracy

INFO:tensorflow:Oracle triggered exit


In [ ]:
#build LSTM model
def build_LSTM_model(hp):
    strategy = tf.distribute.MirroredStrategy()
    with strategy.scope():
        model = Sequential()
        model.add(Embedding(input_dim=num_words,
                    output_dim=300,
                    embeddings_initializer=initializers.Constant(embedding_matrix), 
                    input_length=50,trainable=False))
        model.add(
            Dropout(
                rate=hp.Float(
                    "dropout_1", min_value=0.0, max_value=0.4, default=0.2, step=0.1,
                )
            )
        )
        model.add(            
            LSTM(
                units = 64,
                recurrent_dropout=hp.Float(
                    "recurrent_dropout_2", min_value=0.2, max_value=0.6, default=0.2, step=0.1,
                )
            )
        )
                
        model.add(
            Dense(
                units=1, 
                activation="sigmoid"))

        model.compile(            
            optimizer=keras.optimizers.Adam(
                hp.Float(
                    "learning_rate",
                    min_value=1e-4,
                    max_value=1e-2,
                    sampling="LOG",
                    default=1e-3,
                )
            ),
            loss="binary_crossentropy",
            metrics=["accuracy"])
    return model

In [ ]:
#define and use Hyperband to search optimized parameters
H_tuner_LSTM = Hyperband(
    build_LSTM_model,
    max_epochs=10,
    objective='val_accuracy',
    executions_per_trial=3,
    directory='project_hyperband_LSTM',
    project_name='ML-Project')
H_tuner_LSTM.search(X_train, Y_train,
             epochs=5,
             validation_data=(X_test, Y_test))
Hyperband_summary_LSTM=H_tuner_LSTM.results_summary()
Hyperband_summary_LSTM
with open(r'Hyperband_summary_LSTM.pkl','wb')as f:
    pickle.dump(Hyperband_summary_LSTM,f)

INFO:tensorflow:Reloading Oracle from existing project project_hyperband_LSTM/ML-Project/oracle.json
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
INFO:tensorflow:Reloading Tuner from project_hyperband_LSTM/ML-Project/tuner0.json
INFO:tensorflow:Oracle triggered exit
